In [1]:
import cv2
import face_recognition
from flask import Flask, render_template, Response
import time

In [2]:
#Inicializa a aplicação Flask
app = Flask(__name__)

In [3]:
'''
Inicializa a captura pelo OpenCV

Câmera ip: rtsp://username:password@ip_address:554/user=username_password='password'_channel=channel_number_stream=0.sdp' 
Webcam: cv2.VideoCapture(0)
http://help.angelcam.com/en/articles/372649-finding-rtsp-addresses-for-ip-cameras-nvrs-dvrs
'''

camera = cv2.VideoCapture(0)
anterior = time.time()
fpsLimite = 1 

In [4]:
anterior

1615128378.1381989

In [5]:
agora = time.time()
agora

1615128378.1634567

In [6]:
# Adiciona janela e gera os frames para a câmera

def gen_frames():
    global anterior
    while True:
        sucesso, frame = camera.read()  # Lê frame
        agora = time.time()
        if (agora - anterior) > fpsLimite:
            anterior = agora
            rgb_frame = frame[:, :, ::-1]
            locacao_faces = face_recognition.face_locations(rgb_frame)
            if not sucesso:
                break
            else:            
                for (topo, direita, base, esquerda) in locacao_faces:
                    cv2.rectangle(frame, (esquerda, topo), (direita, base), (0, 0, 255), 2)
                ret, buffer = cv2.imencode('.jpg', frame)
                frame = buffer.tobytes()
                yield (b'--frame\r\n'
                       b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')  # Concatena e mostra resultado

In [7]:
# Define uma rota para a aplicação web

@app.route('/')
def index():
    return render_template('index.html')

In [8]:
# Define uma rota para o stream de video

@app.route('/stream_video')
def stream_video():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

In [9]:
# Inicia o servidor Flask
if __name__ == "__main__":
    app.run(host='127.0.0.1', port=5001, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Mar/2021 10:46:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2021 10:46:24] "GET /stream_video HTTP/1.1" 200 -
